In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import time


In [2]:

# 1. Configuração e Conexão (Simulando um Data Lake)
nome_arquivo = 'vendas_bigdata.csv'
engine = create_engine('sqlite:///data_warehouse.db')

print(f"--- Lendo arquivo: {nome_arquivo} ---")
# Carregamos tudo para memória (No ETL isso é o padrão, no ELT é apenas para envio)
df_raw = pd.read_csv(nome_arquivo)
print(f"Linhas carregadas: {len(df_raw)}")
print(f"Amostra de dados sujos:\n{df_raw.head()}\n")
print (df_raw)

--- Lendo arquivo: vendas_bigdata.csv ---
Linhas carregadas: 10000
Amostra de dados sujos:
   id_venda        data  valor_venda estado_cliente
0      1001  16/09/2024      2344.35             PR
1      1002  15-03-2024      2375.17             SP
2      1003  06/09/2024      1988.13             SC
3      1004  28-02-2024          NaN             SC
4      1005  26-04-2024      2622.69             PR

      id_venda        data  valor_venda estado_cliente
0         1001  16/09/2024      2344.35             PR
1         1002  15-03-2024      2375.17             SP
2         1003  06/09/2024      1988.13             SC
3         1004  28-02-2024          NaN             SC
4         1005  26-04-2024      2622.69             PR
...        ...         ...          ...            ...
9995     10996  08-03-2024      1666.64             rj
9996     10997  2024-02-19      2042.79             BA
9997     10998  19/01/2024      2311.99             SP
9998     10999  01-11-2024      3436.63       

In [3]:

# ==============================================================================
# CENÁRIO 1: ABORDAGEM ETL (Extract -> Transform -> Load)
# O Python é o "operário". Ele processa tudo na memória RAM.
# ==============================================================================
# ==============================================================================
# CENÁRIO 1: ABORDAGEM ETL (Extract -> Transform -> Load)
# ==============================================================================
print(">>> INICIANDO ETL (Processamento via Pandas) <<<")
inicio = time.time()

# A. Transformação
df_etl = df_raw.copy()

# 1. Tratamento de Datas (A CORREÇÃO ESTÁ AQUI)
# Estamos sobrescrevendo a coluna 'data' diretamente
df_etl['data'] = pd.to_datetime(df_etl['data'], dayfirst=True, errors='coerce').dt.strftime('%Y-%m-%d')

# 2. Tratamento de Nulos
df_etl.dropna(subset=['valor_venda'], inplace=True)

# 3. Padronização de Texto
df_etl['estado_cliente'] = df_etl['estado_cliente'].str.upper()

# B. Carga
df_etl.to_sql('tb_vendas_tratadas_etl', engine, if_exists='replace', index=False)

fim = time.time()
# ... resto do código igual
print(f"ETL Concluído em {fim - inicio:.2f} segundos.")
print(f"Linhas finais (ETL): {len(df_etl)}")
print("-" * 30)
# Verificação Final
print("\n--- Verificando resultado no Banco (ETL) ---")
df_final = pd.read_sql(text("SELECT * FROM tb_vendas_tratadas_elt LIMIT 5"), engine)
print(df_final)

>>> INICIANDO ETL (Processamento via Pandas) <<<
ETL Concluído em 0.16 segundos.
Linhas finais (ETL): 9526
------------------------------

--- Verificando resultado no Banco (ETL) ---


DatabaseError: Execution failed on sql 'SELECT * FROM tb_vendas_tratadas_elt LIMIT 5': (sqlite3.OperationalError) no such table: tb_vendas_tratadas_elt
[SQL: SELECT * FROM tb_vendas_tratadas_elt LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:

# ==============================================================================
# CENÁRIO 2: ABORDAGEM ELT (Extract -> Load -> Transform)
# O Python é o "gerente". Ele joga o dado bruto e manda o Banco trabalhar.
# ==============================================================================
print("\n>>> INICIANDO ELT (Processamento via Banco de Dados) <<<")
inicio = time.time()

# A. Carga Imediata (Load Raw) - Jogamos a sujeira para a "Staging Area"
df_raw.to_sql('staging_vendas', engine, if_exists='replace', index=False)

# B. Transformação (Transform) - O Python manda o SQL fazer o trabalho pesado
# Nota: SQLite é limitado em funções de data, mas em bancos reais (Postgres/BigQuery)
# faríamos o parse de data aqui também. Focaremos na limpeza de valores e texto.

query_elt = text ("""
CREATE TABLE tb_vendas_tratadas_elt AS
SELECT 
    id_venda,
    data, 
    UPPER(estado_cliente) as estado_cliente,
    valor_venda
FROM staging_vendas
WHERE valor_venda IS NOT NULL
""")
query_drop=text("""DROP TABLE IF EXISTS tb_vendas_tratadas_elt""")
# Executando a orquestração
with engine.connect() as conn:
    # Remove a tabela antiga se existir (para não dar erro no teste)
    conn.execute(query_drop)
    # Cria a nova tabela processada
    conn.execute(query_elt)

fim = time.time()
print(f"ELT Concluído em {fim - inicio:.2f} segundos.")

# Verificação Final
print("\n--- Verificando resultado no Banco (ELT) ---")
df_final = pd.read_sql("SELECT * FROM tb_vendas_tratadas_elt LIMIT 5", engine)
print(df_final)